<a href="https://colab.research.google.com/github/namtoptall/DataScience/blob/main/transfer_learning_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
!nvidia-smi

Thu Jul 25 20:10:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   72C    P0              33W /  72W |   4319MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Transfer_learning : Upsample

In [68]:
# lay file support functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-07-25 20:10:43--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-07-25 20:10:44 (117 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [69]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir

In [70]:

# Download dữ liệu từ Google Storage (đã được định dạng trước)
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

unzip_data("101_food_classes_10_percent.zip")

train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

--2024-07-25 20:10:44--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.250.141.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip.1’

101_food_classes_10 100%[===================>]   1.51G   202MB/s    in 9.2s    

2024-07-25 20:10:53 (169 MB/s) - ‘101_food_classes_10_percent.zip.1’ saved [1625420029/1625420029]



In [71]:
walk_through_dir("101_food_classes_10_percent")

There are 2 directories and 0 images in '101_food_classes_10_percent'.
There are 101 directories and 0 images in '101_food_classes_10_percent/train'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/deviled_eggs'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/guacamole'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/nachos'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/lobster_roll_sandwich'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/tacos'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/filet_mignon'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/gnocchi'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/beet_salad'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/scallops'.
There are 0 directories and 75 images in '101_foo

In [72]:
# set input
import tensorflow as tf

In [73]:
IMG_SIZE = (224,224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE,
                                                                               shuffle=False)


Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


# Huấn luyện mô hình big dog với transfer learning trên 10% của 101 food classes


In [74]:
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor="val_accuracy",
                                                         save_best_only=True)

In [75]:
# import modules
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

In [76]:
data_augmentation = Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2),
],name = 'data_augmentation')

In [77]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Thiết lập kiến trúc mô hình với các layer trên cùng có thể huấn luyện
inputs = layers.Input(shape=(224, 224, 3), name="input_layer") # shape của ảnh đầu vào
x = data_augmentation(inputs) # tăng cường ảnh (chỉ xảy ra trong quá trình huấn luyện)
x = base_model(x, training=False) # đặt base model trong chế độ suy luận để dùng nó nhằm trích xuất đặc trưng mà không vần cập nhật trọng số
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x) # nhóm đầu ra của base model
outputs = layers.Dense(len(train_data_all_10_percent.class_names), activation="softmax", name="output_layer")(x) # có cùng số lượng đầu ra với các lớp
model = tf.keras.Model(inputs, outputs)


In [78]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_average_pooling (Gl  (None, 1280)              0         
 obalAveragePooling2D)                                           
                                                                 
 output_layer (Dense)        (None, 101)               129381    
                                                           

In [79]:
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(), # sử dụng Adam với các thiết lập mặc định
              metrics=["accuracy"])

# Khớp
history_all_classes_10_percent = model.fit(train_data_all_10_percent,
                                           epochs=10, # khớp cho 5 epoch để giúp các thử nghiệm diễn ra nhanh chóng
                                           validation_data=test_data,
                                           validation_steps=int(0.15 * len(test_data)), # đánh giá trên phần nhỏ hơn của dữ liệu kiểm tra
                                           callbacks=[checkpoint_callback]) # lưu các trọng số mô hình tốt nhất vào file


Epoch 1/10
237/237 [==============================] - 34s 117ms/step - loss: 3.3833 - accuracy: 0.2742 - val_loss: 2.5115 - val_accuracy: 0.4597
Epoch 2/10
237/237 [==============================] - 26s 108ms/step - loss: 2.2074 - accuracy: 0.4972 - val_loss: 2.0617 - val_accuracy: 0.5034
Epoch 3/10
237/237 [==============================] - 26s 108ms/step - loss: 1.8211 - accuracy: 0.5651 - val_loss: 1.9003 - val_accuracy: 0.5289
Epoch 4/10
237/237 [==============================] - 26s 108ms/step - loss: 1.6002 - accuracy: 0.6169 - val_loss: 1.7872 - val_accuracy: 0.5445
Epoch 5/10
237/237 [==============================] - 25s 105ms/step - loss: 1.4588 - accuracy: 0.6421 - val_loss: 1.7694 - val_accuracy: 0.5410
Epoch 6/10
237/237 [==============================] - 24s 99ms/step - loss: 1.3408 - accuracy: 0.6672 - val_loss: 1.7390 - val_accuracy: 0.5434
Epoch 7/10
237/237 [==============================] - 24s 100ms/step - loss: 1.2315 - accuracy: 0.6986 - val_loss: 1.7298 - val_acc

In [ ]:
results_feature_extraction_model = model.evaluate(test_data)
results_feature_extraction_model

In [ ]:
plot_loss_curves(history_all_classes_10_percent)

In [ ]:
base_model.trainable = True

# freeze except 5 last layers
for layer in base_model.layers[:-5]:
     layer.trainable = False

In [ ]:
# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])


In [ ]:
for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
# Kiểm tra xem những layer nào có thể huấn luyện
for layer_number, layer in enumerate(base_model.layers):
  print(layer_number, layer.name, layer.trainable)


In [ ]:
# Tinh chỉnh 5 epoch nữa
fine_tune_epochs = 10 # mô hình đã thực hiện 5 epoch, đây là tổng số epoch mà chúng ta theo đuổi (5+5=10)

history_all_classes_10_percent_fine_tune = model.fit(train_data_all_10_percent,
                                                     epochs=fine_tune_epochs,
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15 * len(test_data)), # xác thực trên 15% dữ liệu kiểm tra
                                                     initial_epoch=history_all_classes_10_percent.epoch[-1]) # bắt đầu từ epoch trước đó



In [ ]:
# Đánh giá mô hình đã tinh chỉnh trên toàn bộ tập kiểm tra
results_all_classes_10_percent_fine_tune = model.evaluate(test_data)
results_all_classes_10_percent_fine_tune

In [ ]:
compare_historys(original_history=history_all_classes_10_percent,
                 new_history=history_all_classes_10_percent_fine_tune,
                 initial_epochs=5)

In [ ]:
# mount drive content
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.save("drive/My Drive/tensorflow_course/101_food_class_10_percent_saved_big_dog_model")


# Đánh giá chất lượng của mô hình big dog trên tất cả các lớp khác nhau


In [ ]:
import tensorflow as tf

# Download pre-trained model từ Google Storage (như chương trình nấu ăn, tối huấn luyện mô hình này trước đó nên các kết quả có thể khác với ở trên)
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/06_101_food_class_10_percent_saved_big_dog_model.zip
saved_model_path = "06_101_food_class_10_percent_saved_big_dog_model.zip"
unzip_data(saved_model_path)

# Lưu ý: việc load mô hình sẽ xuất ra rất nhiều 'WARNINGS', có thể bỏ qua chúng: https://www.tensorflow.org/tutorials/keras/save_and_load#save_checkpoints_during_training
# Cũng có thread trên GitHub sửa các warning này: https://github.com/tensorflow/tensorflow/issues/40166
# model = tf.keras.models.load_model("drive/My Drive/tensorflow_course/101_food_class_10_percent_saved_big_dog_model/") # đường dẫn tới mô hình
model = tf.keras.models.load_model(saved_model_path.split(".")[0]) # không bao gồm ".zip" trong đường dẫn mô hình đã load


In [ ]:
loaded_loss, loaded_accuracy = model.evaluate(test_data)
loaded_loss, loaded_accuracy

In [ ]:
pred_probs = model.predict(test_data, verbose=1) # thiết lập verbose để xem nó tốn bao lâu


In [ ]:
len(pred_probs)

In [ ]:
# Shape của các dự đoán là gì?
pred_probs.shape

In [ ]:
# Chúng trông như thế nào?
pred_probs[:10]

In [ ]:
pred_classes = pred_probs.argmax(axis=1)

# Chúng trông như thế nào?
pred_classes[:10]

In [ ]:
y_labels = []
for images,labels in test_data.unbatch():
  y_labels.append(labels.numpy().argmax())

y_labels[:10]

# Đánh giá các dự đoán của mô hình


In [ ]:
len(y_labels)


In [ ]:
# Lấy hệ số accuracy bằng cách so sánh các lớp đã đoán với nhãn gốc
from sklearn.metrics import accuracy_score
sklearn_accuracy = accuracy_score(y_labels, pred_classes)
sklearn_accuracy

In [ ]:
import numpy as np
print(f"Close? {np.isclose(loaded_accuracy, sklearn_accuracy)} | Difference: {loaded_accuracy - sklearn_accuracy}")

In [ ]:
from helper_functions import make_confusion_matrix

In [ ]:
# Lưu ý: Code confusion matrix sau là bản phối lại của
# hàm plot_confusion_matrix trong Scikit-Learn - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Hàm của chúng ta cần một cái tên khác cho plot_confusion_matrix của sklearn
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False):
  """Tạo một confusion matrix được gấn nhãn so với các đự đoán và nhãn gốc.

  Nếu các class được truyền, confusion matrix sẽ được gắn nhãn; ngược lại, các giá trị integer class
  sẽ được sử dụng.

  Đối số:
    y_true: Mảng các nhãn truth label (phải có cùng shape với y_pred).
    y_pred: Mảng các predicted label (phải có cùng shape với y_true).
    classes: Mảng các class label (ví dụ: dạng string). Nếu `None`, thì sẽ sử dụng integer label.
    figsize: Kích thước của output figure (default=(10, 10)).
    text_size: Kích thước của output figure text (default=15).
    norm: chuẩn hóa các giá trị hoặc không (default=False).
    savefig: lưu confusion matrix vào file (default=False).

  Trả về:
    Biểu đồ confusion matrix được gắn nhãn so với y_true và y_pred.

  Ví dụ:
    make_confusion_matrix(y_true=test_labels, # ground truth test label
                          y_pred=y_preds, # predicted label
                          classes=class_names, # mảng có tên class label
                          figsize=(15, 15),
                          text_size=10)
  """
  # Tạo confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # chuấn hóa nó
  n_classes = cm.shape[0] # tìm số lớp mà chúng ta đang xử lý

  # Vẽ figure và chỉnh cho nó dễ nhìn
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # màu sắc thể hiện mức độ 'đúng' của một lớp, càng tối == càng tốt
  fig.colorbar(cax)

  # Có danh sách các lớp không?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Gắn nhãn các trục
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # tạo đủ axis slot cho mỗi lớp
         yticks=np.arange(n_classes),
         xticklabels=labels, # các trục sẽ được gắn nhãn với tên lớp (nếu chúng tồn tại) hoặc ints
         yticklabels=labels)

  # Khiến các x-axis label xuất hiện ở dưới cùng
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  ### Bổ sung: xoay xticks để đọc và tăng kích thước phông chữ (bắt buộc do confusion matrix lớn như vậy)
  plt.xticks(rotation=70, fontsize=text_size)
  plt.yticks(fontsize=text_size)

  # Đặt ngưỡng cho các màu khác nhau
  threshold = (cm.max() + cm.min()) / 2.

  # Vẽ text ở từng cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Lưu figure vào directory đang làm việc
  if savefig:
    fig.savefig("confusion_matrix.png")


In [ ]:
class_names = test_data.class_names
class_names


In [ ]:
make_confusion_matrix(y_true=y_labels,
                      y_pred=pred_classes,
                      classes=class_names,
                      figsize=(100, 100),
                      text_size=20,
                      norm=False,
                      savefig=True)

### một chút khái niệm về classification_report ⁉
- Precision - Tỷ lệ true positive trên tổng số mẫu. Precision cao hơn dẫn đến ít false positive hơn (mô hình dự đoán 1 trong khi đáng ra phải là 0).
- Recall - Tỷ lệ true positive trên tổng số true positive và false negative (mô hình dự đoán 0 trong khi đáng ra phải là 1). Recall cao hơn dẫn đến ít false negative hơn.
- F1 score - Kết hợp precision và recall thành một phép đo. 1 là tốt nhất, 0 là tệ nhất.


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_labels, pred_classes))

In [ ]:
# Lấy dictionary của classification report
classification_report_dict = classification_report(y_labels, pred_classes, output_dict=True)
classification_report_dict

**Tập trung vào f1_score**

In [ ]:
   # Tạo dictionary rỗng
class_f1_scores = {}
# Lặp qua từng item trong classification report
for k, v in classification_report_dict.items():
    if k == "accuracy": # ngừng lại khi lấy được key accuracy
        break
    else:
    # Thêm tên lớp và f1-score vào dictionary mới
        class_f1_scores[class_names[int(k)]] = v["f1-score"]
class_f1_scores

In [ ]:
import pandas as pd
f1_scores = pd.DataFrame({"class_name": list(class_f1_scores.keys()),
                          "f1-score": list(class_f1_scores.values())}).sort_values("f1-score", ascending=False)
f1_scores

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 25))
scores = ax.barh(range(len(f1_scores)), f1_scores["f1-score"].values)
ax.set_yticks(range(len(f1_scores)))
ax.set_yticklabels(list(f1_scores["class_name"]))
ax.set_xlabel("f1-score")
ax.set_title("F1-Scores for 10 Different Classes")
ax.invert_yaxis(); # đảo ngược thứ tự

def autolabel(rects): # Phiên bản đã sửa của: https://matplotlib.org/examples/api/barchart_demo.html
  """
  Attach a text label above each bar displaying its height (it's value).
  """
  for rect in rects:
    width = rect.get_width()
    ax.text(1.03*width, rect.get_y() + rect.get_height()/1.5,
            f"{width:.2f}",
            ha='center', va='bottom')

autolabel(scores)

In [ ]:
def load_and_prep_image(filename, img_shape=224, scale=True):
  """
  Đọc một ảnh từ filename, biến nó thành một tensor và reshape thành
  (224, 224, 3).

  Các tham số
  ----------
  filename (str): string filename của ảnh mục tiêu
  img_shape (int): kích thước để resize ảnh mục tiêu, mặc định là 224
  scale (bool): có co giãn các giá trị pixel thành range(0, 1) không, mặc định là True
  """
  # Đọc trong ảnh
  img = tf.io.read_file(filename)
  # Giải mã nó thành một tensor
  img = tf.io.decode_image(img)
  # Thay đổi kích thước ảnh
  img = tf.image.resize(img, [img_shape, img_shape])
  if scale:
    # Co giãn ảnh (nhận tất cả các giá trị từ 0 đến 1)
    return img/255.
  else:
    return img

In [ ]:
# Đưa ra dự đoán trên một loạt các ảnh ngẫu nhiên
import os
import random

plt.figure(figsize=(17, 10))
for i in range(3):
  # Chọn một ảnh ngẫu nhiên từ một lớp ngẫu nhiên
  class_name = random.choice(class_names)
  filename = random.choice(os.listdir(test_dir + "/" + class_name))
  filepath = test_dir + class_name + "/" + filename

  # Load ảnh và đưa ra dự đoán
  img = load_and_prep_image(filepath, scale=False) # không co giãn ảnh cho các dự đoán EfficientNet
  pred_prob = model.predict(tf.expand_dims(img, axis=0)) # mô hình chấp nhận các tensor có shape [None, 224, 224, 3]
  pred_class = class_names[pred_prob.argmax()] # tìm lớp được dự đoán

  # Vẽ (các) ảnh
  plt.subplot(1, 3, i+1)
  plt.imshow(img/255.)
  if class_name == pred_class: # Thay đổi màu sắc của text căn cứ vào việc dự đoán đúng hay sai
    title_color = "g"
  else:
    title_color = "r"
  plt.title(f"actual: {class_name}, pred: {pred_class}, prob: {pred_prob.max():.2f}", c=title_color)
  plt.axis(False);

In [ ]:
# 1. Lấy tên file của toàn bộ dữ liệu kiểm tra
filepaths = []
for filepath in test_data.list_files("101_food_classes_10_percent/test/*/*.jpg",
                                     shuffle=False):
  filepaths.append(filepath.numpy())
filepaths[:10]
# 2. Tạo dataframe từ dữ liệu dự đoán hiện tại để phân tích
import pandas as pd
pred_df = pd.DataFrame({"img_path": filepaths,
                        "y_true": y_labels,
                        "y_pred": pred_classes,
                        "pred_conf": pred_probs.max(axis=1), # lấy giá trị xác suất dự đoán lớn nhất
                        "y_true_classname": [class_names[i] for i in y_labels],
                        "y_pred_classname": [class_names[i] for i in pred_classes]})
pred_df.head()

# 3. Dự đoán có đúng không?
pred_df["pred_correct"] = pred_df["y_true"] == pred_df["y_pred"]
pred_df.head()

In [ ]:
# 4. Lấy top 100 mẫu sai sai nhất
top_100_wrong = pred_df[pred_df["pred_correct"] == False].sort_values("pred_conf", ascending=False)[:100]
top_100_wrong.head(20)

In [ ]:
# 5. Hiển thị một số mẫu sai nhất
images_to_view = 9
start_index = 10 # thay đổi start index để thấy nhiều hơn
plt.figure(figsize=(15, 10))
for i, row in enumerate(top_100_wrong[start_index:start_index+images_to_view].itertuples()):
  plt.subplot(3, 3, i+1)
  img = load_and_prep_image(row[1], scale=True)
  _, _, _, _, pred_prob, y_true, y_pred, _ = row # chỉ quan tâm một vài tham số ở mỗi hàng
  plt.imshow(img)
  plt.title(f"actual: {y_true}, pred: {y_pred} \nprob: {pred_prob:.2f}")
  plt.axis(False)


# Kiểm tra mô hình big dog trên hình ảnh kiểm tra cũng như hình ảnh thức ăn tùy chỉnh


In [ ]:
# Lưu ý: chúng ta có thể upload ảnh tùy chỉnh của mình lên Google Colab bằng cách sử dụng nút "upload" trong tab Files
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/custom_food_images.zip

unzip_data("custom_food_images.zip")


In [ ]:
custom_food_images = ["custom_food_images/" + img_path for img_path in os.listdir("custom_food_images")]
custom_food_images


In [ ]:
for img in custom_food_images:
  img = load_and_prep_image(img, scale=False) # load ảnh mục tiêu và biến nó thành tensor
  pred_prob = model.predict(tf.expand_dims(img, axis=0)) # đưa ra dự đoán về hình ảnh với shape [None, 224, 224, 3]
  pred_class = class_names[pred_prob.argmax()] # tìm nhãn của lớp được dự đoán
  # Vẽ hình ảnh với các chú thích phù hợp
  plt.figure()
  plt.imshow(img/255.) # imshow() yêu cầu đầu vào float phải được chuẩn hóa
  plt.title(f"pred: {pred_class}, prob: {pred_prob.max():.2f}")
  plt.axis(False)


# 🛠 Bài tập thực hành
Lưu ý: Các bạn cần làm phần bài tập này để chuẩn bị cho phiên review lab.

- Chọn lấy 3 bức ảnh thức ăn của bạn và sử dụng mô hình đã huấn luyện để đưa ra dự đoán về chúng, chia sẻ dự đoán của bạn với các học viên khác trong Discord và giới thiệu mô hình Food Vision của bạn 🍔👁.
- Huấn luyện mô hình feature-extraction transfer learning cho 10 epoch trong cùng một dữ liệu và so sánh chất lượng của nó với mô hình sử dụng feature extraction cho 5 epoch và fine-tuning cho 5 epoch (như chúng ta đã dùng trong notebook này). Phương pháp nào tốt hơn?
- Tạo lại mô hình đầu tiên (mô hình feature extraction) với mixed_precision.
Nó có giúp mô hình huấn luyện nhanh hơn không?
Nó có ảnh hưởng đến độ chính xác hoặc chất lượng của mô hình không?
Lợi ích của việc sử dụng huấn luyện hỗn mixed_precision là gì?
